<a href="https://colab.research.google.com/github/jelmerbe/ORF401/blob/main/baseball_Pitch_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Baseball Simulation

##Import the data

In [ ]:
## Import (edited) data ##
import numpy as np
import math

#fastball = 0, curveball = 1, slider = 2, changeup = 3
#right = 1, left = 2

data = np.genfromtxt('baseball999.csv', delimiter=',')


##Separate training and testing data

In [ ]:
## Seperate training and testing data ##

train_data = np.zeros([4000,11]) #all the completed rows
test_data = np.zeros([1000,11]) #all the rows with missing data
trainCounter = 0
testCounter = 0
for i in range (0,5000):
  if math.isnan(data[i,3]):
    test_data[testCounter] = data[i]
    testCounter = testCounter + 1
  else :
    train_data[trainCounter] = data[i]
    trainCounter = trainCounter + 1

##Train Data and Build Model

In [ ]:
## Train data and build model ##

#Libraries imported
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

def prediction_model(data):
  #Assign X and y
  X = data[:,[1,2,4,5,6,7,8,9,10]]
  Y = data[:,3]

  X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=0)
    
  d_train=lgb.Dataset(X_train, label=y_train)
  
  #setting up the parameters
  params={}
  params['num_leaves'] = 10
  params['learning_rate']=0.03
  params['boosting_type']='gbdt' #GradientBoostingDecisionTree
  params['objective']='multiclass' #Multi-class target feature
  params['metric']='multi_logloss' #metric for multi-class
  params['max_depth']= 5
  params['num_class']= 4 #no.of unique values in the target class not inclusive of the end value
  params['min_data_in_leaf'] = 10
  params['class_weight'] = 'balanced'

  #training the model
  clf=lgb.train(params,d_train,500)
    
  #prediction on the test dataset
  y = clf.predict(X)
  
  #argmax() method 
  y = [np.argmax(line) for line in y]

  pred = np.zeros(4000)    
  for i in range (0,4000):
    pred = np.round(y, decimals = 0) 
  
  print(pred)
  return pred, clf

    





##Test accuracy of the model

In [ ]:
## Test accuracy of our model ##

count_test = 0
outcome, clf = prediction_model(train_data)

for i in range (4000):
  if outcome[i] == train_data[i, 3]:
    count_test = count_test + 1

#Calculate accuracy 
accuracy = (count_test / 4000) * 100
print("Accuracy percentage is", accuracy, "%")

[0 0 0 ... 0 0 3]
Accuracy percentage is 99.5 %


##Predict missing pitch classifications

In [ ]:
## Predict missing pitch classifications ##
test_X = test_data[:,[1,2,4,5,6,7,8,9,10]]

#Make predictions
Pitch_classifications = clf.predict(test_X)

#argmax() method 
Pitch_classifications = [np.argmax(line) for line in Pitch_classifications]

#Round values
for i in range (0,1000):
  Pitch_classifications = np.round(Pitch_classifications, decimals = 0)

print(Pitch_classifications)

[2 2 1 2 2 0 2 2 0 0 2 2 0 0 2 3 1 2 0 0 2 0 0 0 1 3 1 0 0 2 0 0 2 0 0 2 1
 2 2 2 0 0 0 2 0 0 0 2 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 2 0 0 2 2 0 0 0 0
 0 0 0 0 2 2 0 0 0 0 2 0 1 0 0 0 0 0 0 0 2 2 0 0 0 0 3 0 3 0 0 3 0 0 0 3 3
 0 3 0 3 3 0 2 0 0 3 3 2 2 0 2 3 2 2 0 0 1 0 0 2 1 0 0 2 2 0 3 0 1 3 0 3 0
 0 2 0 0 0 1 0 0 3 0 2 2 2 2 0 2 0 0 3 0 3 3 0 0 2 0 2 2 2 0 1 1 0 0 1 0 0
 1 0 3 1 1 1 3 2 3 3 2 0 1 0 3 2 1 3 0 3 3 0 0 2 0 2 0 0 0 2 2 0 0 2 2 2 0
 2 0 0 2 0 0 3 0 0 2 0 0 2 0 2 0 3 0 0 0 2 0 0 3 0 0 0 0 0 3 0 0 3 0 0 3 2
 2 0 2 0 0 0 0 0 0 0 0 0 0 0 2 0 2 0 2 0 2 0 1 0 0 0 0 2 0 0 1 1 0 1 0 0 0
 1 2 2 3 1 0 0 0 0 0 0 0 1 0 2 0 0 0 0 2 0 1 1 0 2 0 2 2 0 2 0 0 2 2 2 0 0
 0 0 0 2 2 0 1 0 1 0 2 2 0 0 0 1 0 0 0 0 1 0 0 0 2 0 2 0 1 0 2 1 2 1 1 2 2
 1 0 1 0 2 0 2 2 0 0 0 3 0 0 1 0 0 2 2 0 2 0 1 0 1 0 0 2 2 1 0 2 0 1 0 1 2
 2 0 0 2 2 2 0 2 0 2 0 2 0 0 2 1 2 2 1 2 0 1 2 0 1 0 0 0 0 2 0 2 2 0 2 2 0
 0 0 0 2 2 3 2 0 2 2 2 2 1 2 0 0 2 2 0 2 1 0 2 2 0 0 0 2 0 0 0 2 1 0 1 3 2
 0 1 0 0 0 0 2 0 3 2 1 1 

##Export pitch classifications as .csv file

In [ ]:
from numpy import savetxt
## Fill out data set ##
results = np.zeros([100,10])

Counter = 0
for i in range (0,10):
  for j in range(0,100):
    results[j,i] = Pitch_classifications[Counter]
    Counter = Counter + 1

savetxt('results1.csv', results, delimiter=',')